In [4]:
from __future__ import annotations

from pathlib import Path
import importlib

import numpy as np
import pandas as pd

import src.retina_embeddings_dataset as red

# Ensure the notebook uses the latest loader code without needing a kernel restart.
importlib.reload(red)
load_retina_embeddings_dataset = red.load_retina_embeddings_dataset
select_feature_matrix = red.select_feature_matrix

# --- Config (pick ONE dataset + ONE embeddings file) ---
ROOT = Path.cwd()  # expected: .../Sprint-Project-4-5/Solved

DATASET = "mbrset"  # "mbrset" or "brset"

if DATASET == "mbrset":
    EMBEDDINGS_CSV = ROOT / "data" / "mbrset_embeddings" / "Embeddings_vit_base_mbrset.csv"
    LABELS_CSV = ROOT / "data" / "mbrset_embeddings" / "mbrset_labels" / "labels_mbrset.csv"
    VIEW = "macula"  # "macula" keeps .1/.3 images; "all" keeps everything
else:
    EMBEDDINGS_CSV = ROOT / "data" / "brset_embeddings" / "Embeddings_brset_vit_base_.csv"
    LABELS_CSV = ROOT / "data" / "brset_embeddings" / "brset_labels" / "labels_brset.csv"
    VIEW = "all"

# --- Load + merge embeddings with labels; derive tasks ---
ds = load_retina_embeddings_dataset(
    dataset=DATASET,
    embeddings_csv_path=str(EMBEDDINGS_CSV),
    labels_csv_path=str(LABELS_CSV),
    view=VIEW,
)

df = ds.df
print("Merged rows:", len(df))
print("# embedding dims:", len(ds.feature_cols))

# --- Task distributions (clinical + privacy targets) ---
cols_to_show = [
    "task_any_dr",
    "task_referable",
    "task_3class",
    "sex",
]

for c in cols_to_show:
    if c in df.columns:
        vc = df[c].value_counts(dropna=False).sort_index()
        print(f"\n{c} value_counts:\n{vc}")

# --- Feature matrix ---
X = select_feature_matrix(df, ds.feature_cols)
print("\nX shape:", X.shape)

# --- Minimal (numpy-only) baseline: ridge regression classifier ---

def _train_test_split_by_patient(
    df_in: pd.DataFrame,
    patient_col: str,
    test_size: float = 0.2,
    seed: int = 0,
) -> tuple[np.ndarray, np.ndarray]:
    rng = np.random.default_rng(seed)
    patients = df_in[patient_col].dropna().astype(str).unique().tolist()
    patients = np.array(patients, dtype=object)
    rng.shuffle(patients)

    n_test = max(1, int(round(len(patients) * test_size)))
    test_patients = set(patients[:n_test].tolist())

    patient_vals = df_in[patient_col].astype(str).to_numpy()
    test_mask = np.array([p in test_patients for p in patient_vals], dtype=bool)
    train_mask = ~test_mask
    return train_mask, test_mask


def _standardize_fit(X_train: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    mean = X_train.mean(axis=0)
    std = X_train.std(axis=0)
    std[std == 0] = 1.0
    return mean, std


def _standardize_apply(X_in: np.ndarray, mean: np.ndarray, std: np.ndarray) -> np.ndarray:
    return (X_in - mean) / std


def _ridge_fit(X_train: np.ndarray, y_train: np.ndarray, alpha: float = 1.0) -> np.ndarray:
    # Adds bias term.
    Xb = np.hstack([X_train, np.ones((X_train.shape[0], 1), dtype=X_train.dtype)])
    # Closed-form: w = (X^T X + a I)^-1 X^T y
    d = Xb.shape[1]
    A = Xb.T @ Xb + alpha * np.eye(d, dtype=Xb.dtype)
    b = Xb.T @ y_train
    w = np.linalg.solve(A, b)
    return w


def _ridge_predict_proba(X_in: np.ndarray, w: np.ndarray) -> np.ndarray:
    Xb = np.hstack([X_in, np.ones((X_in.shape[0], 1), dtype=X_in.dtype)])
    logits = Xb @ w
    # sigmoid
    return 1.0 / (1.0 + np.exp(-logits))


def _f1_binary(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    tp = int(((y_true == 1) & (y_pred == 1)).sum())
    fp = int(((y_true == 0) & (y_pred == 1)).sum())
    fn = int(((y_true == 1) & (y_pred == 0)).sum())
    if tp == 0:
        return 0.0
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall = tp / (tp + fn) if (tp + fn) else 0.0
    if (precision + recall) == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)


def run_binary_task(task_col: str, patient_col: str = "patient_id") -> None:
    if task_col not in df.columns:
        print(f"\n[skip] Missing column: {task_col}")
        return
    if patient_col not in df.columns:
        print(f"\n[skip] Missing patient column: {patient_col}")
        return

    task_df = df.dropna(subset=[task_col]).copy()
    y = task_df[task_col].astype(int).to_numpy()
    X_task = select_feature_matrix(task_df, ds.feature_cols)

    train_mask, test_mask = _train_test_split_by_patient(task_df, patient_col=patient_col, seed=0)
    X_train, X_test = X_task[train_mask], X_task[test_mask]
    y_train, y_test = y[train_mask], y[test_mask]

    mean, std = _standardize_fit(X_train)
    X_train_s = _standardize_apply(X_train, mean, std)
    X_test_s = _standardize_apply(X_test, mean, std)

    w = _ridge_fit(X_train_s, y_train.astype(np.float32), alpha=10.0)
    p = _ridge_predict_proba(X_test_s, w)
    y_pred = (p >= 0.5).astype(int)

    acc = float((y_pred == y_test).mean())
    f1 = _f1_binary(y_test, y_pred)

    print(f"\nTask: {task_col}")
    print("test n:", len(y_test), "pos rate:", float(y_test.mean()) if len(y_test) else None)
    print("accuracy:", round(acc, 4), "f1:", round(f1, 4))


run_binary_task("task_any_dr")
run_binary_task("task_referable")


Merged rows: 2582
# embedding dims: 768

task_any_dr value_counts:
task_any_dr
0    2001
1     581
Name: count, dtype: Int64

task_referable value_counts:
task_referable
0    2131
1     451
Name: count, dtype: Int64

task_3class value_counts:
task_3class
0       1856
1        478
2        103
<NA>     145
Name: count, dtype: Int64

sex value_counts:
sex
0    1680
1     902
Name: count, dtype: int64

X shape: (2582, 768)

Task: task_any_dr
test n: 516 pos rate: 0.23062015503875968
accuracy: 0.405 f1: 0.4107

Task: task_referable
test n: 516 pos rate: 0.18410852713178294
accuracy: 0.3992 f1: 0.3595


In [5]:
# --- Proper models (GroupKFold by patient) ---
import importlib

import src.retina_evaluation as reval

importlib.reload(reval)

# Choose which tasks to run
TASKS = ["task_any_dr", "task_referable"]  # add "task_3class" later
MODELS = ["logreg", "rf", "mlp", "xgb", "lgbm"]

all_summaries = []
all_oof = []

for task in TASKS:
    res = reval.evaluate_binary_models_groupkfold(
        df=df,
        feature_cols=ds.feature_cols,
        label_col=task,
        group_col="patient_id",
        model_names=MODELS,
        n_splits=5,
        seed=0,
    )
    s = res.summary.copy()
    s.insert(0, "task", task)
    all_summaries.append(s)
    all_oof.append(res.oof)

summary_df = pd.concat(all_summaries, ignore_index=True)
oof_df = pd.concat(all_oof, ignore_index=True)

print("\n=== Cross-validated summary (patient-level GroupKFold) ===")
display(summary_df)

# --- Fairness / subgroup slices (sex + age bins) ---
# Uses out-of-fold predictions so we slice metrics on held-out data.
for task in TASKS:
    print(f"\n=== Subgroup report for: {task} ===")
    task_oof = oof_df[oof_df["label_col"] == task]

    # Pick best model by roc_auc_mean (fallback to first if all NaN)
    best = (
        summary_df[summary_df["task"] == task]
        .sort_values(by="roc_auc_mean", ascending=False, na_position="last")
        .iloc[0]["model"]
    )
    print("Best model:", best)

    if "sex" in task_oof.columns and task_oof["sex"].notna().any():
        sex_report = reval.fairness_report_binary(oof=task_oof, model_name=best, by="sex")
        print("\nBy sex:")
        display(sex_report)

    if "age" in task_oof.columns and task_oof["age"].notna().any():
        age_report = reval.fairness_report_binary(oof=task_oof, model_name=best, by="age_bin")
        print("\nBy age_bin (<40 / 40-60 / >60):")
        display(age_report)


KeyboardInterrupt: 